Importing all required dependencies

In [1]:
from itertools import product
from collections import defaultdict
import numpy as np
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cosine
import pulp
import gensim
from sklearn.metrics.pairwise import cosine_similarity
import json
import pickle


Importing Word2Vec model (https://code.google.com/archive/p/word2vec/). The next block may require a long time!

In [2]:
wvmodel = gensim.models.KeyedVectors.load_word2vec_format(r'GoogleNews-vectors-negative300.bin', binary=True)

Testing Word2Vec model (@Mario: Nota quanto cazzo è incredibile sta cosa)

In [3]:
x_test = wvmodel['man']+wvmodel['king']-wvmodel['woman']
y_test = wvmodel['queen']
z_test = wvmodel['boat']

In [4]:
print(cosine_similarity([x_test], [y_test]))
print(cosine_similarity([x_test], [z_test]))

[[ 0.40576041]]
[[ 0.01503608]]


Useful functions

In [5]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [6]:
def dot_product_lists(a,b):
    return sum([x*y for x,y in zip(a,b)])

In [7]:
def wme_to_wmd(x,y,gamma):
    kxy = dot_product_lists(x,y)
    return -1/gamma*np.log(kxy)

Implementation of Word Mover Distance (solved through LP). Next two blocks from: https://github.com/stephenhky/PyWM

In [8]:
def tokens_to_fracdict(tokens):
    cntdict = defaultdict(lambda : 0)
    for token in tokens:
        cntdict[token] += 1
    totalcnt = sum(cntdict.values())
    return {token: float(cnt)/totalcnt for token, cnt in cntdict.items()}

In [9]:
# UBER MEGA FAST WMD BY AL

from gensim.corpora.dictionary import Dictionary
def wmdistance(document1, document2, wvmodel):
        from pyemd import emd

        # Remove out-of-vocabulary words.
        len_pre_oov1 = len(document1)
        len_pre_oov2 = len(document2)
        remember_me = document2
        document2 = ['__qxvca^&3fd?#_!$' +  str(i) for i in range(len(document2))]
        diff1 = len_pre_oov1 - len(document1)
        diff2 = len_pre_oov2 - len(document2)
        if diff1 > 0 or diff2 > 0:
            logger.info('Removed %d and %d OOV words from document 1 and 2 (respectively).', diff1, diff2)

        if not document1 or not document2:
            logger.info(
                "At least one of the documents had no words that were in the vocabulary. "
                "Aborting (returning inf)."
            )
            return float('inf')

        dictionary = Dictionary(documents=[document1, document2])
        vocab_len = len(dictionary)

        if vocab_len == 1:
            # Both documents are composed by a single unique token
            return 0.0

        # Sets for faster look-up.
        docset1 = set(document1)
        docset2 = set(document2)

        wordvecs_random = {document2[i]: remember_me[i] for i in range(len(remember_me))}

        # Compute distance matrix.
        distance_matrix = np.zeros((vocab_len, vocab_len))
        for i, t1 in dictionary.items():
            if t1 not in docset1:
                continue

            for j, t2 in dictionary.items():
                if t2 not in docset2 or distance_matrix[i, j] != 0.0:
                    continue

                # Compute Euclidean distance between word vectors.
                distance_matrix[i, j] = distance_matrix[j, i] = np.sqrt(sum((wvmodel[t1] - wordvecs_random[t2])**2))

        def nbow(document):
            d = np.zeros(vocab_len)
            nbow = dictionary.doc2bow(document)  # Word frequencies.
            doc_len = len(document)
            for idx, freq in nbow:
                d[idx] = freq / float(doc_len)  # Normalized word frequencies.
            return d

        # Compute nBOW representation of documents.
        d1 = nbow(document1)
        d2 = nbow(document2)

        # Compute WMD.
        return emd(d1, d2, distance_matrix)

In [10]:
def word_mover_distance_probspec(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=None):
    all_tokens = list(set(first_sent_tokens+second_sent_tokens))
    wordvecs = {token: wvmodel[token] for token in all_tokens}

    first_sent_buckets = tokens_to_fracdict(first_sent_tokens)
    second_sent_buckets = tokens_to_fracdict(second_sent_tokens)

    T = pulp.LpVariable.dicts('T_matrix', list(product(all_tokens, all_tokens)), lowBound=0)

    prob = pulp.LpProblem('WMD', sense=pulp.LpMinimize)
    prob += pulp.lpSum([T[token1, token2]*euclidean(wordvecs[token1], wordvecs[token2])
                        for token1, token2 in product(all_tokens, all_tokens)])
    for token2 in second_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token1 in first_sent_buckets])==second_sent_buckets[token2]
    for token1 in first_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token2 in second_sent_buckets])==first_sent_buckets[token1]

    if lpFile!=None:
        prob.writeLP(lpFile)

    prob.solve()

    return prob

In [11]:
def WMD_regular(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=None):
    prob = word_mover_distance_probspec(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=lpFile)
    return pulp.value(prob.objective)

We need to define WMD also for a random document with random words

In [12]:
def word_mover_distance_random(documents_token, random_document_vectors, wvmodel, lpFile=None):
    #We create a random association between the random_document_vectors and tokens, assuming random vectors to be all different
    first_sent_tokens = documents_token
    second_sent_tokens = ['__qxvca^&3fd?#_!$' +  str(i) for i in range(len(random_document_vectors))]
    
    all_tokens = list(set(first_sent_tokens+second_sent_tokens))
    wordvecs_document = {token: wvmodel[token] for token in first_sent_tokens}
    wordvecs_random = {second_sent_tokens[i]: random_document_vectors[i] for i in range(len(random_document_vectors))}
    #print(wordvecs_random)
    wordvecs = merge_two_dicts(wordvecs_document,wordvecs_random)

    first_sent_buckets = tokens_to_fracdict(first_sent_tokens)
    second_sent_buckets = tokens_to_fracdict(second_sent_tokens)

    T = pulp.LpVariable.dicts('T_matrix', list(product(all_tokens, all_tokens)), lowBound=0)

    prob = pulp.LpProblem('WMD', sense=pulp.LpMinimize)
    prob += pulp.lpSum([T[token1, token2]*euclidean(wordvecs[token1], wordvecs[token2])
                        for token1, token2 in product(all_tokens, all_tokens)])
    for token2 in second_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token1 in first_sent_buckets])==second_sent_buckets[token2]
    for token1 in first_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token2 in second_sent_buckets])==first_sent_buckets[token1]

    if lpFile!=None:
        prob.writeLP(lpFile)

    prob.solve()

    return prob

In [13]:
def WMD_random(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=None):
    prob = word_mover_distance_random(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=lpFile)
    return pulp.value(prob.objective)

Test WMD functions

In [14]:
print(WMD_regular(['hi','bye'],['see','you'],wvmodel))
print(WMD_random(['hi','bye'],[wvmodel['see'],wvmodel['you']],wvmodel))
print(wmdistance(['hi','bye'],wvmodel['see','you'],wvmodel))

3.61462414265
3.61462414265
3.61462488407


Implementation of the WME 

In [15]:
def WME_phi(x,w,gamma,wvmodel):
    return np.exp( -gamma*wmdistance(x,w,wvmodel))

In [16]:
def calculate_WME(documents, D_max, R, wvmodel, gamma):
    # It must returns a list of text embeddings, i-th element being the embedding of the i-th document
    
    #Phase 1: Compute v_max and v_min
    v_values = [+9999,-9999] # [v_min,v_max]
    for doc in documents:
        for token in doc:
            for x in wvmodel[token]:
                v_values[0] = min(v_values[0],x)
                v_values[1] = max(v_values[1],x)
            
    print("[v_min, v_max] = "  + str(v_values))
    Z = []
    for j in range(R):
        print("R: " + str(j+1)+"/"+str(R))
        D = 1 + np.random.randint(D_max)
        random_doc = []
        for l in range(D):
            word = np.random.uniform(v_values[0],v_values[1],size=300)
            word = word*1/np.sqrt(sum(x*x for x in word))
            random_doc.append(word)
        to_add = [WME_phi(doc,random_doc,gamma,wvmodel) for doc in documents]
        #print("to_add" + str(to_add))
        Z.append(to_add)
    ret_Z = (1/(np.sqrt(R))*np.array(Z).T).tolist()
    return ret_Z
        
        
    

In [17]:
documents = [['the','sun','is','the','best','thing','in','the','world'],['i','love','watch','the','sun','it','burns','my','life']]

In [18]:
#It returns a list of k_max elements, iteratively picked using the farthest-first traversal algorithm
# The returned list [a_1, ..., a_(k_max)] containts the indexes of the picked element at every iteration (i.e. a_i is selected in the i-th iteration)
# documents in this case containts the (ordered) list of the embeddings of the documents
def k_center(documents,k_max):
    ret = []
    radius = []
    N = len(documents)
    first = np.random.randint(N)
    ret.append(first)
    
    dist = [euclidean(doc,documents[first]) for doc in documents] 
   
    radius.append(np.max(dist))
    for k in range(k_max-1):
        #Selection
        j = np.argmax(dist)
        ret.append(j)
        
        #Update
        dist = [min(dist[i], euclidean(documents[i],documents[j])) for i in range(N)]
       
        radius.append(np.max(dist))
    return ret,radius
        
        
        
    

import pickle
f = open('newsgroup_dataset.pckl','rb')
obj = pickle.load(f)
f.close()
name_dataset = obj[0]
train_R_X = obj[1]
train_R_Y = obj[2]

CARE! The next line of code is computationally expensive! (Disabled)

train_embedding = calculate_WME(train_R_X,6,128,wvmodel)

The next block can be used to save the trained model (Put it as code)

import pickle 
obj = ['newsgroup_WME-D6-R128-N1500-L25', train_R_X, train_R_Y, train_embedding] 
f = open('newsgroup_WME-D6-R128-N1500-L25.pckl', 'wb') 
pickle.dump(obj, f) 
f.close()

import pickle
obj = ['recipe_dataset', train_R_X, train_R_Y, index_to_pick, train_embedding]
f = open('recipes_WME.pckl', 'wb')
pickle.dump(obj, f)
f.close()

The next block can be used to load the trained model (Put it as code)

import pickle
f = open('newsgroup_WME-D6-R128-N1500-L25.pckl', 'rb')
obj = pickle.load(f)
f.close()
print(len(obj))
name_dataset = obj[0]
train_R_X = obj[1]
train_R_Y = obj[2]
train_embedding = obj[3]
print(name_dataset)

In [73]:
elements_to_pick = 150
NUMBER_OF_POINTS = len(train_R_X)
centers,radius = k_center(train_embedding,elements_to_pick)

In [74]:
sets_label = set()
k_center_number = []
for i in range(elements_to_pick):
    sets_label.add(train_R_Y[centers[i]])
    k_center_number.append(len(sets_label))
    

In [75]:
random_indexes = [i for i in range(NUMBER_OF_POINTS)]
np.random.shuffle(random_indexes)
sets_label = set()
random_number = [0]*elements_to_pick
NUMBER_ITERATIONS = 100000 #Reduce if too much expensive
for j in range(NUMBER_ITERATIONS):
    sets_label = set()
    for i in range(150):
        sets_label.add(train_R_Y[random_indexes[i]])
        random_number[i] += 1.0/NUMBER_ITERATIONS*len(sets_label)

In [19]:
import matplotlib.pyplot as plt

In [20]:
from sklearn import metrics

In [52]:
def calculate_ARI(dataset, centers, labels):
    dict_labels = {}
    ii = 0
    for x in set(labels):
        if x not in dict_labels:
            dict_labels[x] = ii
            ii = ii+1
    true_labels = [dict_labels[x] for x in labels]
    found_labels = []
    for x in dataset:
        j = np.argmin([euclidean(x,c) for c in centers])
        found_labels.append(j)
    print(true_labels)
    print(found_labels)
    return [metrics.adjusted_rand_score(true_labels, found_labels),metrics.adjusted_mutual_info_score(true_labels,found_labels)]
    

In [21]:
print(train_R_Y)

['tennis' 'football' 'cricket' 'cricket' 'football' 'rugby' 'rugby'
 'football' 'athletics' 'cricket' 'athletics' 'cricket' 'tennis'
 'athletics' 'rugby' 'tennis' 'rugby' 'athletics' 'tennis' 'cricket'
 'athletics' 'cricket' 'athletics' 'cricket' 'cricket' 'tennis' 'cricket'
 'football' 'football' 'cricket' 'athletics' 'cricket' 'tennis' 'cricket'
 'rugby' 'athletics' 'football' 'cricket' 'athletics' 'tennis' 'athletics'
 'cricket' 'football' 'cricket' 'cricket' 'athletics' 'football'
 'athletics' 'athletics' 'rugby' 'athletics' 'athletics' 'athletics'
 'tennis' 'rugby' 'rugby' 'tennis' 'football' 'tennis' 'rugby' 'athletics'
 'rugby' 'cricket' 'cricket' 'athletics' 'rugby' 'athletics' 'athletics'
 'football' 'rugby' 'athletics' 'athletics' 'athletics' 'football'
 'football' 'cricket' 'rugby' 'tennis' 'athletics' 'tennis' 'cricket'
 'football' 'cricket' 'tennis' 'athletics' 'athletics' 'rugby' 'tennis'
 'cricket' 'athletics' 'rugby' 'tennis' 'cricket' 'cricket' 'football'
 'tennis' 'te

f = open('bbcsport_embeddingsR=512_gamma=1p12', 'wb')
obj = ['bbcsport_embeddings', all_train_embeddings, train_R_Y, cutoff, gammas, ] 
pickle.dump(obj, f) 
f.close()

In [21]:
import pickle
f = open('bbcsport_dataset.pckl','rb')
obj = pickle.load(f)
f.close()
name_dataset = obj[0]
train_R_X = obj[1]
train_R_Y = obj[2]

In [24]:
import sys
sys.path.append('./liblinear-2.21/python/')
from liblinearutil import *

In [25]:
jj = 0
to_number = {}
for x in set(train_R_Y):
    to_number[x] = jj
    jj = jj + 1
int_train_R_Y = [to_number[x] for x in train_R_Y]

In [26]:
import pickle

CV = 10
R_values = [ 128, 256 ]
D_values = [ 6 ]
gamma_values = [ 0.1 ]
N = len(train_R_X)

for R_val in R_values:
    for D_val in D_values:
        for gamma_val in gamma_values: 
            print("Using R = " + str(R_val) + ", D = " + str(D_val) + ", gamma = " + str(gamma_val))
            embedding_points = calculate_WME(train_R_X,D_val,R_val,wvmodel,gamma_val)
            best_accuracy = -1
            best_lambda = 'boh'
            best_std = 0
            lamda_inverse = ['1e2', '3e2', '5e2', '8e2', '1e3', '3e3', '5e3', '8e3', '1e4', '3e4', '5e4', '8e4', '1e5' ,'3e5' ,'8e5' ,'1e6', '3e6', '7e6' ,'1e7' ,'5e7' ,'3e8' ,'1e9']
            for lambda_val in lamda_inverse:
                accuracy_vector = []
                kf = KFold(n_splits=10)
                kf.get_n_splits(embedding_points)
                for train_index, test_index in kf.split(embedding_points):
                    X_train = np.array(embedding_points)[train_index]
                    X_test = np.array(embedding_points)[test_index]
                    
                    Y_train = np.array(int_train_R_Y)[train_index]
                    Y_test = np.array(int_train_R_Y)[test_index]
                    
                    prob = problem(Y_train,X_train)
                    param = parameter('-s 2 -e 0.0001 -q -c ' + lambda_val)
                    m = train(prob,param)
                    p_label, p_acc, p= predict(Y_test, X_test, m)
                    ACC, MSE, SCC = evaluations(Y_test, p_label)
                    accuracy_vector.append(ACC)
                avg_ACC = np.mean(accuracy_vector)
                std_ACC = np.std(accuracy_vector)
                if(avg_ACC > best_accuracy):
                    
                    best_accuracy = avg_ACC
                    best_lambda = lambda_val
                    best_std = std_ACC
            
            print("R = " + str(R_val) + " D = " + str(D_val) + " gamma = " + str(gamma_val) + " accuracy = " + str(best_accuracy))
            #We need to save datas here
            f = open('increasingR_newsgroup_WME-R'+str(R_val)+"-D"+str(D_val)+"-gamma"+str(gamma_val)+ '.pckl','wb')
            obj = [train_R_X,D_val,R_val,gamma_val,embedding_points,best_accuracy, best_std,best_lambda]
            pickle.dump(obj, f) 
            f.close()

            

                    
                    

Using R = 128, D = 6, gamma = 0.1
[v_min, v_max] = [-1.171875, 1.3203125]
R: 1/128
R: 2/128
R: 3/128
R: 4/128
R: 5/128
R: 6/128
R: 7/128
R: 8/128
R: 9/128
R: 10/128
R: 11/128
R: 12/128
R: 13/128
R: 14/128
R: 15/128
R: 16/128
R: 17/128
R: 18/128
R: 19/128
R: 20/128
R: 21/128
R: 22/128
R: 23/128
R: 24/128
R: 25/128
R: 26/128
R: 27/128
R: 28/128
R: 29/128
R: 30/128
R: 31/128
R: 32/128
R: 33/128
R: 34/128
R: 35/128
R: 36/128
R: 37/128
R: 38/128
R: 39/128
R: 40/128
R: 41/128
R: 42/128
R: 43/128
R: 44/128
R: 45/128
R: 46/128
R: 47/128
R: 48/128
R: 49/128
R: 50/128
R: 51/128
R: 52/128
R: 53/128
R: 54/128
R: 55/128
R: 56/128
R: 57/128
R: 58/128
R: 59/128
R: 60/128
R: 61/128
R: 62/128
R: 63/128
R: 64/128
R: 65/128
R: 66/128
R: 67/128
R: 68/128
R: 69/128
R: 70/128
R: 71/128
R: 72/128
R: 73/128
R: 74/128
R: 75/128
R: 76/128
R: 77/128
R: 78/128
R: 79/128
R: 80/128
R: 81/128
R: 82/128
R: 83/128
R: 84/128
R: 85/128
R: 86/128
R: 87/128
R: 88/128
R: 89/128
R: 90/128
R: 91/128
R: 92/128
R: 93/128
R: 94

NameError: name 'KFold' is not defined

In [177]:
np.std([1,2,3])

0.81649658092772603

In [184]:
print(best_std)

1.86794326841
